<a href="https://colab.research.google.com/github/Summer99D/MLF_Final/blob/main/MLF_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from category_encoders import TargetEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_trns_df= pd.read_csv('/content/drive/MyDrive/MLF/test_transaction.csv')
test_id_df= pd.read_csv('/content/drive/MyDrive/MLF/test_identity.csv')
train_trns_df= pd.read_csv('/content/drive/MyDrive/MLF/train_transaction.csv')
train_id_df= pd.read_csv('/content/drive/MyDrive/MLF/train_identity.csv')


In [ ]:
print(test_trns_df.describe)

In [ ]:
print(test_id_df.describe)

In [ ]:
print(train_id_df.describe)
print(train_id_df.columns)

In [ ]:
print(train_trns_df.describe)
print(train_trns_df.columns)

In [ ]:
## I will be merging on "transaction id"
train_merged= pd.merge(train_trns_df, train_id_df, on='TransactionID', how='left')
test_merged= pd.merge(test_trns_df, test_id_df, on='TransactionID', how='left')

In [ ]:
print(train_merged.shape)## we have 434 variables
print(test_merged.shape)

### Inspecting NaNs

Training data

In [ ]:
# Step 1: Calculate NaN statistics
nan_percentage_train = train_merged.isnull().mean() * 100
nan_percentage_train = nan_percentage_train.sort_values(ascending=False)

# Step 2: Summary statistics
print("NaN Distribution Across Columns:")
print(f"Total rows: {train_merged.shape[0]}")
print(f"Total columns: {train_merged.shape[1]}")
print(f"Columns with >30% missing values: {(nan_percentage_train > 30).sum()}") ##removing columns with more than 30% missing vlaues
print("\nTop 10 columns with highest missing values:")
print(nan_percentage_train.head(10))


In [ ]:
# Plot 1: Bar plot of NaN percentages (top 50 columns)
plt.figure(figsize=(15, 8))
nan_percentage_train.head(50).plot(kind='bar')
plt.title('NaN % Before Preprocessing (Top 50 Columns)')
plt.xlabel('Column name')
plt.ylabel('Missing Value %')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/nan_bar_plot_before.png')
plt.show()

In [ ]:
print(nan_percentage_train.describe)

In [ ]:
# Remove columns with >30% missing values
columns_to_drop = nan_percentage_train[nan_percentage_train > 30].index
print(f"Removing {len(columns_to_drop)} columns with >30% missing values:")
print(columns_to_drop.tolist())
train_data_cleaned = train_merged.drop(columns=columns_to_drop)

In [ ]:
## I wanna check data types
train_data_cleaned.dtypes

In [ ]:
# Impute NaNs in all columns with the mean (for numeric columns only)
train_data_cleaned = train_data_cleaned.fillna(train_data_cleaned.mean(numeric_only=True))

In [ ]:
from sklearn.impute import SimpleImputer

# Identify column types
num_cols = train_data_cleaned.select_dtypes(include=['float64', 'int64']).columns
cat_cols = train_data_cleaned.select_dtypes(include=['object', 'category']).columns

num_imputer = SimpleImputer(strategy='mean')
train_data_cleaned[num_cols] = num_imputer.fit_transform(train_data_cleaned[num_cols])


cat_imputer = SimpleImputer(strategy='most_frequent')
train_data_cleaned[cat_cols] = cat_imputer.fit_transform(train_data_cleaned[cat_cols])


In [ ]:
# Verify no NaNs
nan_percentage_cleaned = train_data_cleaned.isnull().mean() * 100
print("\nNaN percentages after imputation (should be 0%):")
print(nan_percentage_cleaned[nan_percentage_cleaned > 0])

Test Data

In [ ]:
# Step 1: Calculate NaN statistics
nan_percentage_test = test_merged.isnull().mean() * 100
nan_percentage_test = nan_percentage_test.sort_values(ascending=False)

# Step 2: Summary statistics
print("NaN Distribution Across Columns:")
print(f"Total rows: {test_merged.shape[0]}")
print(f"Total columns: {test_merged.shape[1]}")
print(f"Columns with >30% missing values: {(nan_percentage_test > 30).sum()}") ##removing columns with more than 30% missing vlaues
print("\nTop 10 columns with highest missing values:")
print(nan_percentage_test.head(10))

In [ ]:
# Bar plot: Top 50 columns
plt.figure(figsize=(15, 8))
nan_percentage_test.head(50).plot(kind='bar')
plt.title('NaN % Before Preprocessing (Top 50 Columns)')
plt.xlabel('Column name')
plt.ylabel('Missing Value %')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/nan_bar_plot_before.png')
plt.show()

In [ ]:
# Remove columns with >30% missing values
columns_to_drop_test = nan_percentage_test[nan_percentage_test > 30].index
print(f"Removing {len(columns_to_drop_test)} columns with >30% missing values:")
print(columns_to_drop_test.tolist())
test_data_cleaned = test_merged.drop(columns=columns_to_drop_test)

In [ ]:
# Impute NaNs in all columns with the mean (for numeric columns only)
test_data_cleaned = test_data_cleaned.fillna(test_data_cleaned.mean(numeric_only=True))

## Running PCA

In [ ]:
# Step 3: Prepare for PCA
X = train_data_cleaned.drop(columns=['isFraud', 'TransactionID'])
y = train_data_cleaned['isFraud']


In [ ]:
##encoding them!
print("\nCategorical Columns:", cat_cols.tolist())


In [ ]:
# One-hot encode non-numerical columns
print("\nApplying one-hot encoding to non-numerical columns")
print(f"Non-numerical columns: {cat_cols.tolist()}")
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True, sparse=True)
X_encoded = X_encoded.sparse.to_coo().tocsr()
print("Encoded data shape:", X_encoded.shape)
print("Sample encoded columns:", X_encoded.columns[:10].tolist())